In [16]:
import numpy as np
from scipy.stats import norm, t
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

A randomized, double-blind experiment was conducted to assess the
effectiveness of several drugs for reducing postoperative nausea. The
data are as follows. (read from file)

(a) Test each drug versus the placebo at the 5 per cent level. Also, report
the estimated odds–ratios. Summarize your findings.
(b) Use the Bonferroni and the FDR method to adjust for multiple
testing. (Beecher (1959)).

In [17]:
data = pd.read_excel('../data/nausea.xlsx', index_col = False, engine = 'openpyxl')
data.index = data['Treatment'].values
data

,Treatment,Number of Patients,Incidence of Nausea
Placebo,Placebo,80,45
Chlorpromazine,Chlorpromazine,75,26
Dimenhydrinate,Dimenhydrinate,85,52
Pentobarbital (100mg),Pentobarbital (100mg),67,35
Pentobarbital (150mg),Pentobarbital (150mg),85,37


In [18]:
data['p^'] = data['Incidence of Nausea'] / data['Number of Patients']
data

,Treatment,Number of Patients,Incidence of Nausea,p^
Placebo,Placebo,80,45,0.562500
Chlorpromazine,Chlorpromazine,75,26,0.346667
Dimenhydrinate,Dimenhydrinate,85,52,0.611765
Pentobarbital (100mg),Pentobarbital (100mg),67,35,0.522388
Pentobarbital (150mg),Pentobarbital (150mg),85,37,0.435294


In [19]:
data['se^'] = data['p^'] * (1 - data['p^']) / data['Number of Patients']
data

,Treatment,Number of Patients,Incidence of Nausea,p^,se^
Placebo,Placebo,80,45,0.562500,0.003076
Chlorpromazine,Chlorpromazine,75,26,0.346667,0.003020
Dimenhydrinate,Dimenhydrinate,85,52,0.611765,0.002794
Pentobarbital (100mg),Pentobarbital (100mg),67,35,0.522388,0.003724
Pentobarbital (150mg),Pentobarbital (150mg),85,37,0.435294,0.002892


In [20]:
p_hat_placebo = data.loc['Placebo', 'p^']
se_hat_placebo = data.loc['Placebo', 'se^']

In [21]:
data['W'] = (data['p^'] - p_hat_placebo) / np.sqrt(data['se^'] + se_hat_placebo)
data

,Treatment,Number of Patients,Incidence of Nausea,p^,se^,W
Placebo,Placebo,80,45,0.562500,0.003076,0.000000
Chlorpromazine,Chlorpromazine,75,26,0.346667,0.003020,-2.764364
Dimenhydrinate,Dimenhydrinate,85,52,0.611765,0.002794,0.642987
Pentobarbital (100mg),Pentobarbital (100mg),67,35,0.522388,0.003724,-0.486428
Pentobarbital (150mg),Pentobarbital (150mg),85,37,0.435294,0.002892,-1.646605


In [22]:
data['p-value'] = 2 * norm.cdf(-np.abs(data['W']))
data

,Treatment,Number of Patients,Incidence of Nausea,p^,se^,W,p-value
Placebo,Placebo,80,45,0.562500,0.003076,0.000000,1.000000
Chlorpromazine,Chlorpromazine,75,26,0.346667,0.003020,-2.764364,0.005703
Dimenhydrinate,Dimenhydrinate,85,52,0.611765,0.002794,0.642987,0.520232
Pentobarbital (100mg),Pentobarbital (100mg),67,35,0.522388,0.003724,-0.486428,0.626664
Pentobarbital (150mg),Pentobarbital (150mg),85,37,0.435294,0.002892,-1.646605,0.099639


In [23]:
alpha = 0.05
data['Rejected Regular'] = data['p-value'] < alpha
data

,Treatment,Number of Patients,Incidence of Nausea,p^,se^,W,p-value,Rejected Regular
Placebo,Placebo,80,45,0.562500,0.003076,0.000000,1.000000,False
Chlorpromazine,Chlorpromazine,75,26,0.346667,0.003020,-2.764364,0.005703,True
Dimenhydrinate,Dimenhydrinate,85,52,0.611765,0.002794,0.642987,0.520232,False
Pentobarbital (100mg),Pentobarbital (100mg),67,35,0.522388,0.003724,-0.486428,0.626664,False
Pentobarbital (150mg),Pentobarbital (150mg),85,37,0.435294,0.002892,-1.646605,0.099639,False


In [24]:
bonferroni_threshold = alpha / 4
data['Rejected Bonferroni'] = data['p-value'] < bonferroni_threshold
data

,Treatment,Number of Patients,Incidence of Nausea,p^,se^,W,p-value,Rejected Regular,Rejected Bonferroni
Placebo,Placebo,80,45,0.562500,0.003076,0.000000,1.000000,False,False
Chlorpromazine,Chlorpromazine,75,26,0.346667,0.003020,-2.764364,0.005703,True,True
Dimenhydrinate,Dimenhydrinate,85,52,0.611765,0.002794,0.642987,0.520232,False,False
Pentobarbital (100mg),Pentobarbital (100mg),67,35,0.522388,0.003724,-0.486428,0.626664,False,False
Pentobarbital (150mg),Pentobarbital (150mg),85,37,0.435294,0.002892,-1.646605,0.099639,False,False


In [25]:
ordered_pvalues = np.sort(data['p-value'][1:])
ordered_pvalues

array([0.00570339, 0.09963923, 0.52023237, 0.62666409])

In [26]:
# p-values are independent
# recheck this to make sure

Cm = 1
m = len(ordered_pvalues)
l = [i*alpha / (Cm * m) for i in range(1, m+1)]
l

[0.0125, 0.025, 0.037500000000000006, 0.05]

In [27]:
R = 0
rejected_fdr = [False] # for the placebo
for i in range(m):
    if ordered_pvalues[i] < l[i]:
        R = i+1
        rejected_fdr.append(True)
    else:
        rejected_fdr.append(False)

print(R)
print(rejected_fdr)

1
[False, True, False, False, False]


In [28]:
data['Rejected FDR'] = rejected_fdr
data

,Treatment,Number of Patients,Incidence of Nausea,p^,se^,W,p-value,Rejected Regular,Rejected Bonferroni,Rejected FDR
Placebo,Placebo,80,45,0.562500,0.003076,0.000000,1.000000,False,False,False
Chlorpromazine,Chlorpromazine,75,26,0.346667,0.003020,-2.764364,0.005703,True,True,True
Dimenhydrinate,Dimenhydrinate,85,52,0.611765,0.002794,0.642987,0.520232,False,False,False
Pentobarbital (100mg),Pentobarbital (100mg),67,35,0.522388,0.003724,-0.486428,0.626664,False,False,False
Pentobarbital (150mg),Pentobarbital (150mg),85,37,0.435294,0.002892,-1.646605,0.099639,False,False,False


In [29]:
summary = data.iloc[1:]
summary

,Treatment,Number of Patients,Incidence of Nausea,p^,se^,W,p-value,Rejected Regular,Rejected Bonferroni,Rejected FDR
Chlorpromazine,Chlorpromazine,75,26,0.346667,0.003020,-2.764364,0.005703,True,True,True
Dimenhydrinate,Dimenhydrinate,85,52,0.611765,0.002794,0.642987,0.520232,False,False,False
Pentobarbital (100mg),Pentobarbital (100mg),67,35,0.522388,0.003724,-0.486428,0.626664,False,False,False
Pentobarbital (150mg),Pentobarbital (150mg),85,37,0.435294,0.002892,-1.646605,0.099639,False,False,False
